# Companies Queries

Cuaderno-guía para probar y depurar consultas sobre la colección `companies` en MongoDB. Incluye ejemplos ejecutables, referencias rápidas y notas de interpretación para que terceros entiendan qué devuelve cada helper.

**Navegación rápida**
- Conexión y variables base
- Slugs
- Primary Domain
- DataSources: URLs
- DataSources: Links
- DataSources: Textos
- DataSources: Role / Kind

## Conectarse a la DB y definir ejemplos

Usa `get_db()` para abrir la base por defecto (usa credenciales de `.env`). Ajusta `slug` y `datasource_url` para replicar los ejemplos con otra compañía.

In [ ]:
from src.DB.mongo import get_db

#conectar MongoDB
db = get_db()

#Seleccionar Companies
companies = db["companies"]

#Inputs de ejemplo
slug = "wbuild"
datasource_url = "https://wbuild.io"

## Basic Retrieval
Funciones básicas para obtener documentos directamente por su slug o por una URL de dataSource específica.

**Qué hace cada helper**
- `get_company_by_slug`: retorna el documento completo de una compañía.
- `get_datasource_by_url`: retorna específicamente el sub-documento del dataSource que coincide con la URL.


In [ ]:
from src.DB.companies_querys import get_company_by_slug, get_datasource_by_url

# Obtener documento completo de la compañía
company_doc = get_company_by_slug(slug)

# Obtener dataSource específico (usando proyección posicional $ por defecto)
datasource_doc = get_datasource_by_url(slug, datasource_url)

print(f"Company encontrada: {company_doc.get('name') if company_doc else 'No encontrada'}")
print(f"DataSource encontrado: {'Sí' if datasource_doc else 'No'}")
for i in datasource_doc["dataSources"]:
    print(i)

## Slugs
Funciones para listar, deduplicar y detectar slugs repetidos en `companies`.

**Qué hace cada helper**
- `get_all_slugs`: retorna todos los slugs (puede incluir vacíos y repetidos).
- `get_unique_slugs`: usa `distinct` para quedarte solo con valores únicos.
- `get_repeated_slugs`: lista slugs duplicados con su `count`, ordenados desc.

In [ ]:
from src.DB.companies_querys import (
    get_all_slugs, 
    get_unique_slugs, 
    get_repeated_slugs
    )

slugs = get_all_slugs() #get_all_slugs(include_empty: bool = False) -> List[str] Retorna todos los slugs en la colección, puede incluir repetidos
slugs_unique = get_unique_slugs() #get_unique_slugs(include_empty: bool = False) -> List[str]  Retorna slugs únicos usando distinct
slugs_repetidos = get_repeated_slugs()  #get_repeated_slugs(include_empty: bool = False) -> List[Dict[str, Any]]  Retorna slugs repetidos con count, ordenados por count desc

In [ ]:
print("get_all_slugs (incluye repetidos)")
print(slugs)
print(len(slugs))
print("")
print("get_unique_slugs (unicos)")
print(slugs_unique)
print(len(slugs_unique))
print("")
print("get_repeated_slugs (slugs repetidos)")
print(slugs_repetidos)
print(len(slugs_repetidos))

## Primary Domain
Funciones para listar y controlar el campo `primaryDomain`.

**Qué hace cada helper**
- `get_all_primary_domains`: devuelve todos los `primaryDomain` (incluye vacíos / repetidos si existen).
- `get_unique_primary_domains`: mismos valores pero únicos vía `distinct`.
- `get_repeated_primary_domains`: duplicados con su `count`, ordenados desc.

In [ ]:
from src.DB.companies_querys import (
    get_all_primary_domains,
    get_unique_primary_domains,
    get_repeated_primary_domains,
)

primary_domains = get_all_primary_domains()  # get_all_primary_domains(include_empty: bool = False) -> List[str] Retorna todos los primaryDomain, puede incluir repetidos
primary_domains_unique = get_unique_primary_domains()  # get_unique_primary_domains(include_empty: bool = False) -> List[str] Retorna primaryDomain unicos usando distinct
primary_domains_repetidos = get_repeated_primary_domains()  # get_repeated_primary_domains(include_empty: bool = False) -> List[Dict[str, Any]] Retorna primaryDomain repetidos con count, ordenados por count desc


In [ ]:
print("get_all_primary_domains (incluye repetidos)")
print(primary_domains)
print(len(primary_domains))
print("")

print("get_unique_primary_domains (unicos)")
print(primary_domains_unique)
print(len(primary_domains_unique))
print("")

print("get_repeated_primary_domains (primaryDomain repetidos)")
print(primary_domains_repetidos)
print(len(primary_domains_repetidos))

## DataSources
Funciones para inspeccionar URLs, links y textos almacenados en `dataSources` del modelo. Cada subsección muestra el helper, parámetros clave y cómo interpretar el resultado.

### URLs: extracción y verificación
Obtiene URLs únicas, repetidas y filtradas por `primaryDomain` para un `slug` concreto.

#### `get_unique_datasource_urls`
- Input: `slug`
- Output: lista de `dataSources.url` únicos para esa company.
- Útil para un vistazo rápido a las URLs que ya tenemos cargadas.

In [ ]:
from src.DB.companies_querys import get_unique_datasource_urls

datasource_urls_unique = get_unique_datasource_urls(slug)  # get_unique_datasource_urls(slug: str) -> List[str] Retorna urls unicas de dataSources.url para la company

In [ ]:
print("get_unique_datasource_urls (urls unicas en dataSources.url)")
print(datasource_urls_unique)
print(len(datasource_urls_unique))

#### `get_repeated_datasource_urls`
- Input: `slug`
- Output: URLs repetidas con `count`.
- Útil para detectar scraping duplicado o datos mal normalizados.

In [ ]:
from src.DB.companies_querys import get_repeated_datasource_urls

datasource_urls_repeated = get_repeated_datasource_urls(slug)  # get_repeated_datasource_urls(slug: str) -> List[Dict[str, Any]] Retorna urls repetidas en dataSources.url con count

In [ ]:
print("get_repeated_datasource_urls (urls repetidas en dataSources.url con count)")
print(datasource_urls_repeated)
print(len(datasource_urls_repeated))

#### `unique_company_urls_from_primary_domain`
Compara `dataSources.url` con `primaryDomain` y retorna las URLs válidas para esa compañía.
- `mode="loose"`: acepta subdominios y variaciones que contengan el label base.
- `mode="strict"`: solo primaryDomain exacto y subdominios directos.

In [ ]:
from src.DB.companies_querys import unique_company_urls_from_primary_domain

urls_loose = unique_company_urls_from_primary_domain(slug, mode="loose")  # unique_company_urls_from_primary_domain(slug: str, mode: str = "loose") -> List[str] URLs unicas del dataSources que matchean primaryDomain usando modo loose
urls_strict = unique_company_urls_from_primary_domain(slug, mode="strict")  # unique_company_urls_from_primary_domain(slug: str, mode: str = "loose") -> List[str] URLs unicas del dataSources que matchean primaryDomain usando modo strict


In [ ]:
print("unique_company_urls_from_primary_domain")
print("mode=loose Incluye primaryDomain y subdominios. Tambien acepta casos tipo ejemplo.algo.com si contiene el label base del primaryDomain")
print(urls_loose)
print(len(urls_loose))
print("")
print("mode=strict Solo incluye primaryDomain y subdominios directos. No acepta casos tipo ejemplo.algo.com si no es subdominio del primaryDomain")
print(urls_strict)
print(len(urls_strict))

### Links: navegación dentro de cada URL
Extrae links guardados en `dataSources.links` filtrando por secciones de la página.

#### `get_links_from_company_datasource`
- `sections`: `None` (head, header, main, footer), string o lista de secciones.
- Devuelve links únicos por `datasource_url` para analizar estructura de navegación.

In [ ]:
from src.DB.companies_querys import get_links_from_company_datasource

# get_links_from_company_datasource(slug: str, datasource_url: str, sections=None) -> List[str]
# Retorna links unicos desde dataSources.links para una url especifica dentro de una company.
# sections controla desde que partes se extraen links:
# - None: head header main footer
# - "main": solo esa seccion
# - ["header", "main"]: combinatoria

links_all = get_links_from_company_datasource(slug, datasource_url)  # sections=None
links_main = get_links_from_company_datasource(slug, datasource_url, sections="main")
links_footer = get_links_from_company_datasource(slug, datasource_url, sections="footer")
links_combo = get_links_from_company_datasource(slug, datasource_url, sections=[ "main", "footer"])

In [ ]:
print("get_links_from_company_datasource sections=None (head header main footer)")
print(links_all)
print(len(links_all))
print("")

print("get_links_from_company_datasource sections=main (solo main)")
print(links_main)
print(len(links_main))
print("")

print("get_links_from_company_datasource sections=head (solo head)")
print(links_footer)
print(len(links_footer))
print("")

print("get_links_from_company_datasource sections=[ main, footer] (combinatoria)")
print(links_combo)
print(len(links_combo))

### Textos: contenido extraído de cada URL
Recupera los textos ya parseados desde `dataSources.texts` para auditar calidad de scraping.

#### `get_texts_from_company_datasource`
- `sections`: `None`, string o lista para escoger head/header/main/footer.
- `dedupe`: controla si se deduplican los textos.
- Sirve para revisar si el contenido extraído es coherente antes de procesar embeddings.

In [ ]:
from src.DB.companies_querys import get_texts_from_company_datasource

# get_texts_from_company_datasource(slug: str, datasource_url: str, sections=None, dedupe: bool = True) -> List[str]
# Retorna textos desde dataSources.texts para una url especifica dentro de una company.
# sections controla desde que partes se extraen textos:
# - None: head header main footer
# - "main": solo esa seccion
# - ["header", "main"]: combinatoria
# dedupe controla si se deduplica el output:
# - True: lista unica sin duplicados
# - False: lista tal cual sin deduplicar

texts_all = get_texts_from_company_datasource(slug, datasource_url)  # sections=None dedupe=True
texts_main = get_texts_from_company_datasource(slug, datasource_url, sections="main")  # dedupe=True
texts_header = get_texts_from_company_datasource(slug, datasource_url, sections="header")  # dedupe=True
texts_combo_no_dedupe = get_texts_from_company_datasource(slug, datasource_url, sections=["header", "main", "footer"], dedupe=False)

In [ ]:
print("get_texts_from_company_datasource sections=None dedupe=True (head header main footer)")
print(texts_all)
print(len(texts_all))
print("")

print("get_texts_from_company_datasource sections=main dedupe=True (solo main)")
print(texts_main)
print(len(texts_main))
print("")

print("get_texts_from_company_datasource sections=header dedupe=True (solo header)")
print(texts_header)
print(len(texts_header))
print("")

print("get_texts_from_company_datasource sections=[header, main, footer] dedupe=False (combinatoria sin deduplicar)")
print(texts_combo_no_dedupe)
print(len(texts_combo_no_dedupe))

### Role: etiquetar URLs según su función
Permite leer, asignar o borrar la etiqueta `role` asociada a cada `datasource_url` (ej: `official_site`, `blog`, `careers`).

#### `datasource_role`
- `action`: `get`, `set`, `delete`.
- `role`: requerido solo en `set`.
- Útil para normalizar qué URL consideramos oficial o secundaria.

In [ ]:
from src.DB.companies_querys import datasource_role

current_role = datasource_role(slug, datasource_url, action="get")  # datasource_role(slug: str, datasource_url: str, action: str = "get", role: Optional[str] = None) -> Union[str, None, Dict[str, int]]
print("datasource_role action=get (role actual)")
print(current_role)

In [ ]:
set_result = datasource_role(slug, datasource_url, action="set", role="official_site")
print("datasource_role action=set role=official_site (resultado update)")
print(set_result)

In [ ]:
current_role_after_set = datasource_role(slug, datasource_url, action="get")
print("datasource_role action=get (role despues de set)")
print(current_role_after_set)

In [ ]:
delete_result = datasource_role(slug, datasource_url, action="delete")
print("datasource_role action=delete (resultado unset)")
print(delete_result)

In [ ]:
current_role_after_delete = datasource_role(slug, datasource_url, action="get")
print("datasource_role action=get (role despues de delete)")
print(current_role_after_delete)

### Kind: clasificar el tipo de URL
Gestiona el campo `kind` para cada `datasource_url` (ej: `website`, `social`, `marketplace`).

#### `datasource_kind`
- `action`: `get`, `set`, `delete`.
- `kind`: requerido en `set`.
- Útil para depurar contenido duplicado o separar dominios sociales/landing.

In [ ]:
from src.DB.companies_querys import datasource_kind

current_kind = datasource_kind(slug, datasource_url, action="get")  # datasource_kind(slug: str, datasource_url: str, action: str = "get", kind: Optional[str] = None) -> Union[str, None, Dict[str, int]]
print("datasource_kind action=get (kind actual)")
print(current_kind)


In [ ]:
set_result = datasource_kind(slug, datasource_url, action="set", kind="website")
print("datasource_kind action=set kind=website (resultado update)")
print(set_result)

In [ ]:
current_kind_after_set = datasource_kind(slug, datasource_url, action="get")
print("datasource_kind action=get (kind despues de set)")
print(current_kind_after_set)

In [ ]:
delete_result = datasource_kind(slug, datasource_url, action="delete")
print("datasource_kind action=delete (resultado unset)")
print(delete_result)

In [ ]:
current_kind_after_delete = datasource_kind(slug, datasource_url, action="get")
print("datasource_kind action=get (kind despues de delete)")
print(current_kind_after_delete)